<a href="https://colab.research.google.com/github/cianryan09/VaR-Calculation/blob/main/PRMIA_Exercise_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1. Construct your portfolio

df_returns = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PRM/PRM Returns.csv', index_col='date')
df_returns = df_returns.fillna(0)

In [ ]:
df_weights = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PRM/PRM Weights.csv', index_col=False)
df_weights

In [ ]:
weights = df_weights['Weight %'].to_numpy()
weights

array([8.6500e-03, 3.7000e-03, 1.8100e-03, 6.1800e-03, 7.8600e-03,
       1.0490e-02, 4.9600e-03, 1.3400e-03, 4.5400e-03, 9.6700e-03,
       2.8000e-04, 3.1400e-03, 1.2800e-03, 9.1500e-03, 1.3170e-02,
       1.5000e-03, 1.5700e-03, 8.7400e-03, 1.3600e-03, 3.7900e-03,
       7.0000e-04, 3.8600e-03, 1.5000e-04, 3.7100e-03, 1.4840e-02,
       3.7000e-03, 8.1700e-03, 1.9268e-01, 1.7500e-03, 1.5300e-03,
       6.6900e-03, 9.9000e-04, 4.0000e-03, 1.5400e-03, 6.0000e-04,
       6.7070e-02, 1.2228e-01, 2.0280e-02, 6.8500e-03, 3.9300e-03,
       3.4300e-03, 4.0461e-01, 2.7400e-03, 1.0200e-03, 1.4100e-03,
       9.1600e-03, 9.1300e-03])

In [ ]:
start_dates = ['2019-01-02', '2020-01-02', '2021-01-04', '2022-01-03', '2023-01-03']
end_dates = ['2019-12-31', '2020-12-31', '2021-12-31', '2022-12-30', '2023-12-29']

In [ ]:
dates = df_returns.index.to_list()
dates

In [ ]:
starts = [dates.index(s) for s in start_dates]
ends = [dates.index(e) for e in end_dates]

indices = [(s, e) for s, e in zip(starts, ends)]
indices

[(0, 251), (252, 504), (505, 756), (757, 1007), (1008, 1257)]

In [ ]:
returns = df_returns.to_numpy().T
# returns_arrays = [returns[:,s:e] for (s,e) in indices]
# covariances = [np.cov(ret) for ret in returns_arrays]
# covariances

In [ ]:
# 3yr Daily VaR - Normal Distribution

year = ends[2]
rolling_window = ends[-1] - year
# rolling_window = 6

returns_arrays = [returns[:, n : year + n] for n in range(rolling_window)]
covariances = [np.cov(ret) for ret in returns_arrays]

In [ ]:
std_devs = np.array([np.sqrt(weights @ c @ weights.T) for c in covariances])


In [ ]:
import scipy.stats as st
z_score_95 = st.norm.ppf(.95)
z_score_99 = st.norm.ppf(.99)

In [ ]:
# 2a. VaR Estimation

VaR_95 = std_devs * z_score_95
VaR_99 = std_devs * z_score_99

In [ ]:
VaR_95.shape

(501,)

In [ ]:
years_diffs = [b - a for (a, b) in indices]
years_diffs

[251, 252, 251, 250, 249]

In [ ]:
# 2b. EMWA VaR

# sig = 0.94

# alphas = [np.array([(1 - sig) /(1 - sig ** (-n)) for n in range(-e,0)]) for e in years_diffs]
# alphas[0]

In [ ]:
# 2b. EMWA VaR

sig = 0.94

alphas = np.array([(1 - sig) /(1 - sig ** (-n)) for n in range(-year,0)])
alphas.shape

(756,)

In [ ]:
# emwa_returns = [np.multiply(a, r) for a, r in zip(alphas, returns_arrays)]

# emwa_covs = [np.cov(r, aweights=a) for r, a in zip(returns_arrays, alphas)]
# emwa_covs

In [ ]:
# emwa_returns = [np.multiply(a, r) for a, r in zip(alphas, returns_arrays)]

emwa_covs = [np.cov(r, aweights=alphas) for r in returns_arrays]
len(emwa_covs)

501

In [ ]:
emwa_std_devs = np.array([np.sqrt(weights @ c @ weights.T) for c in emwa_covs])
emwa_std_devs

In [ ]:
emwa_var_95 = emwa_std_devs * z_score_95
emwa_var_99 = emwa_std_devs * z_score_99

In [ ]:
returns_arrays[0].shape

(47, 756)

In [ ]:
# historical VaR

weighted_returns = [weights.T @ r for r in returns_arrays]
weighted_returns[0].shape

(756,)

In [ ]:
hist_var_95 = np.array([np.percentile(np.array(w), .05) for w in weighted_returns])
hist_var_95

array([-0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01649987,
       -0.01649987, -0.01649987, -0.01649987, -0.01649987, -0.01

In [ ]:
hist_var_99 = np.array([np.percentile(w, .01) for w in weighted_returns])
hist_var_99

array([-0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01694658,
       -0.01694658, -0.01694658, -0.01694658, -0.01694658, -0.01

In [ ]:
rolling_window = returns[:, year + 1 : ]
p_returns = weights.T @ rolling_window
p_returns

In [ ]:
counter = 0
for r, v in zip(p_returns, VaR_95):
#  print('My return is: {} and my VaR is {}. backtest exception is {}'.format(r, -v, (r < -v)))
 counter += (r < -v)

print(counter)

72


In [ ]:
norm_var_99_exceptions = np.array([(r < v) for r, v in zip(p_returns, hist_var_99)]).sum()
norm_var_99_exceptions

4

In [ ]:
# back testing

# norm var

norm_var_95_exceptions = np.array([(r < -v) for r, v in zip(p_returns, VaR_95)]).sum()
norm_var_99_exceptions = np.array([(r < -v) for r, v in zip(p_returns, VaR_99)]).sum()


#emwa var

emwa_var_95_exceptions = np.array([(r < -v) for r, v in zip(p_returns, emwa_var_95)]).sum()
emwa_var_99_exceptions = np.array([(r < -v) for r, v in zip(p_returns, emwa_var_99)]).sum()

# hist var

hist_var_95_exceptions = np.array([(r < v) for r, v in zip(p_returns, hist_var_95)]).sum()
hist_var_99_exceptions = np.array([(r < v) for r, v in zip(p_returns, hist_var_99)]).sum()

In [ ]:
exceptions = np.array([norm_var_95_exceptions, norm_var_99_exceptions, emwa_var_95_exceptions, emwa_var_99_exceptions, hist_var_95_exceptions, hist_var_99_exceptions])
exceptions

array([72, 31, 64, 26,  4,  4])

In [ ]:
new_index = df_returns.index[year + 1:]
new_index2 = df_returns.index[year + 1:]

In [ ]:
len(new_index)

501

In [ ]:
# df_exceptions = pd.DataFrame(exceptions, columns = ['norm_var_95_exceptions', 'norm_var_99_exceptions', 'emwa_var_95_exceptions', 'emwa_var_99_exceptions', 'hist_var_95_exceptions', 'hist_var_99_exceptions'], index=[str(e) for e in exceptions])
df_exceptions = pd.DataFrame(exceptions, columns = ['# exceptions'], index=['norm_var_95_exceptions', 'norm_var_99_exceptions', 'emwa_var_95_exceptions', 'emwa_var_99_exceptions', 'hist_var_95_exceptions', 'hist_var_99_exceptions'])
df_exceptions

,# exceptions
norm_var_95_exceptions,72
norm_var_99_exceptions,31
emwa_var_95_exceptions,64
emwa_var_99_exceptions,26
hist_var_95_exceptions,4
hist_var_99_exceptions,4


In [ ]:
vols_and_vars = np.array([std_devs, VaR_95, VaR_99, emwa_std_devs, emwa_var_95, emwa_var_99, abs(hist_var_95), abs(hist_var_99)])

In [ ]:
df_vars = pd.DataFrame(vols_and_vars.T, columns = ['std_devs', 'VaR_95', 'VaR_99', 'emwa_std_devs', 'emwa_var_95', 'emwa_var_99', 'hist_var_95', 'hist_var_99'], index=new_index2)

In [ ]:
df_exceptions.to_csv('Backtest Exceptions.csv', index='date')

In [ ]:
df_vars.to_csv('VaR Results.csv', index='date')